# Cone search in target list using HEALpix indices

In [3]:
import pandas as pd
import astropy_healpix as hpx

In [10]:
from astropy.coordinates import SkyCoord, ICRS
from astropy import units as u

In [19]:
df = pd.read_csv('C:\APT_Images\Processed images\Observing & Processing information.csv')

In [20]:
df = df.dropna(subset = ['RA'])

In [11]:
hp = hpx.HEALPix(nside=16, frame = ICRS())

In [21]:
# calculate HEALPix pixel for a dataframe row

def pixel(row):
    c = SkyCoord(row['RA'], row['Dec'], frame = 'icrs')
    return pd.Series([hp.skycoord_to_healpix(c)], index = ['pixel'])

In [22]:
pixels = df.apply(pixel, axis = 1)

In [23]:
df = pd.concat([df, pixels], axis = 1)

In [24]:
df

,Imaging Date,Start Time,Target,Type,Caldwell,RA,Dec,Dist (ly),ISO,Lights,Darks,Flats,Filename,Processing Date,Processing notes,Notes,pixel
0,22/10/2020,21:09,Altair,Star,NaN,19h50m46.99855s,+08d52m05.9563s,16.7,1600,1 x 20s,NaN,NaN,Altair 2020-10-22,11/12/2020,"Darktable, save jpg",First session with EOS 600D attached to telesc...,1365
1,22/10/2020,21:06,Deneb,Star,NaN,20h41m26s,+45d16m49s,2615.0,1600,1 x 20s,NaN,NaN,Deneb 2020-10-22,11/12/2020,"Darktable, save jpg",- No guiding,471
2,22/10/2020,21:00,Vega,Star,NaN,18h36m56.33635s,+38d47m01.2802s,25.0,800,1 x 20s,NaN,NaN,Vega 2020-10-22,6/12/2020,"Darktable, save jpg",- Experimenting with different ISO,594
3,5/11/2020,19:49,Albireo,Star,NaN,19h30m43.286s,+27d57m34.84s,430.0,800,1 x 20s,NaN,NaN,Albireo 2020-11-05,5/11/2020,"Darktable, save jpg",First use of APT slewing and platesolving,852
4,5/11/2020,19:58,Perseus double cluster,Cluster,C14,02h20m0.0s,+57d08m0.0s,7500.0,800,30 x 60s,10 x 60s,NaN,perseus double cluster,20/11/2020,"DSS, Darktable",First stacked image,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,23/3/22,20:51,NGC2403,Galaxy,C7,07h36m51.4s,+65d36m09s,9650000.0,800,40 x 90s,NaN,NaN,NaN,NaN,NaN,NaN,155
71,24/3/22,20:35,M64 Black Eye galaxy,Galaxy,NaN,12h56m43.696s,+21d40m57.57s,17300000.0,800,40 x 90s,NaN,NaN,NaN,NaN,NaN,NaN,1026
72,24/3/22,21:56,NGC7023 Iris nebula,Nebula,C4,21h01m35.60s,+68d10m10.0s,1300.0,800,40 x 90s,NaN,NaN,NaN,NaN,NaN,NaN,108
73,25/3/22,22:00,M109,Galaxy,NaN,11h57m36.0s,+53d22m28s,83500000.0,800,40 x 90s,NaN,NaN,NaN,NaN,NaN,NaN,337


In [25]:
search_coord = SkyCoord('02h20m0.0s', '+57d08m0.0s', frame = 'icrs')

In [30]:
def cone_search(df, coord, radius):
    pixels = hp.cone_search_skycoord(coord, radius)
    return df[df['pixel'].isin(pixels)]

In [34]:
cone_search(df, search_coord, 15 * u.deg)

,Imaging Date,Start Time,Target,Type,Caldwell,RA,Dec,Dist (ly),ISO,Lights,Darks,Flats,Filename,Processing Date,Processing notes,Notes,pixel
4,5/11/2020,19:58,Perseus double cluster,Cluster,C14,02h20m0.0s,+57d08m0.0s,7500.0,800,30 x 60s,10 x 60s,NaN,perseus double cluster,20/11/2020,"DSS, Darktable",First stacked image,224
11,17/12/2020,18:46,IC1805 Heart Nebula,Nebula,NaN,02h33m22s,+61d26m36s,7500.0,800,40 x 60s,10 x 60s,NaN,ic1805 heart nebula,31/12/2020,"DSS, Startools",NaN,184
50,03/10/2021,20:46,IC342 Hidden galaxy,Galaxy,C5,03h46m48.5s,+68d05m46s,10700000.0,800,40 x 90s,10 x 90s,31,ic342-2021-10-03,20/1/22,NaN,NaN,88
54,21/1/2022,20:08,ngc281 Pacman nebula,Nebula,NaN,00h52m59.3s,+56d37m19s,9500.0,800,25 x 90s,10 x 90s,31,NaN,NaN,NaN,Battery/power management issues - not visible ...,221
64,18/3/22,21:23,NGC869,Cluster,C14,02h19m06s,+57d09m00s,7500.0,800,38 x 90s,NaN,NaN,NaN,NaN,NaN,NaN,224
65,18/3/22,19:57,NGC891 Outer Limits galaxy,Galaxy,C23,02h22m33.4s,+42d20m57s,27300000.0,800,21 x 90s,NaN,NaN,NaN,NaN,NaN,NaN,486
